In [3]:
import os
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import requests
import networkx as nx
from networkx.drawing.nx_agraph import graphviz_layout
from tqdm.auto import tqdm
tqdm.pandas()
from IPython.display import display, HTML
import subprocess

# Parse ENA data table

This data was gathered from ENA on 2024-02-13 by using the "Advanced Search" with the following options:

    Data Type: Raw Reads
        Query: tax_tree(2) AND instrument_platform="oxford_nanopore"
       Fields: run_accession,experiment_title,tax_id,instrument_platform,base_count,experiment_accession,experiment_alias,instrument_model,isolate,library_construction_protocol,library_gen_protocol,library_layout,library_name,library_selection,library_source,library_strategy,pcr_isolation_protocol,project_name,protocol_label,read_count,run_alias,sample_accession,sample_title,sample_alias,sample_description,sample_material,sequencing_method,sra_bytes,sra_ftp,study_accession,study_title,scientific_name,sub_strain,taxonomic_classification,taxonomic_identity_marker,submitted_format,submitted_bytes,submitted_ftp,submission_accession

In [4]:
df = pd.read_csv('../download/2024-02-13_ENA_results_read_run.tsv', sep="\t", low_memory=False)

In [5]:
# only keep entries with .tar.gz data (likely containing raw fast5 files)
df = df.loc[(df.submitted_format == 'OXFORDNANOPORE_NATIVE') & 
             df.submitted_ftp.str.contains('.tar.gz')]
df = df.loc[~df.submitted_ftp.str.lower().str.contains('fastq')]
print(len(df), "ENA runs with .tar.gz ftp data")

947 ENA runs with .tar.gz ftp data


In [6]:
# get lineage information from ENA taxonomy id
#def get_lineage(row):
#    try:
#        return requests.get(f"https://www.ebi.ac.uk/ena/taxonomy/rest/tax-id/{row.tax_id}?binomialOnly=true").json()['lineage']
#    except:
#        return None
def get_lineage(row):
    # retrieve current lineage from ncbi 
    # esearch -db taxonomy -query "Providencia alcalifaciens" | efetch -format xml | 
    # xtract -pattern Taxon -element TaxId -element ScientificName -element Lineage
    cmd1 = ["efetch", "-db", "taxonomy", "-format", "xml", "-id", f"{row.tax_id}"]
    cmd2 = ["xtract", "-pattern", "Taxon", "-element", "Lineage"]
    p1 = subprocess.Popen(cmd1, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    p2 = subprocess.Popen(cmd2, stdin=p1.stdout, stdout=subprocess.PIPE)
    out, err = p2.communicate()
    out = out.decode().strip()
    #print(out)
    if not out:
        return None
    if ";" not in out:
        return None
    return out

df['lineage'] = df.progress_apply(get_lineage, axis=1)
df['lineage'] = df['lineage'].str.rstrip().str.rstrip(';')

  0%|          | 0/947 [00:00<?, ?it/s]

In [7]:
df

,run_accession,experiment_title,tax_id,instrument_platform,base_count,experiment_accession,experiment_alias,instrument_model,isolate,library_construction_protocol,...,study_title,scientific_name,sub_strain,taxonomic_classification,taxonomic_identity_marker,submitted_format,submitted_bytes,submitted_ftp,submission_accession,lineage
835,ERR10447791,MinION sequencing,562,OXFORD_NANOPORE,0,ERX9972173,ena-EXPERIMENT-TAB-01-11-2022-07:51:02:527-10527,MinION,SC419,NaN,...,E. coli methylation,Escherichia coli,NaN,NaN,NaN,OXFORDNANOPORE_NATIVE,21338903437,ftp.sra.ebi.ac.uk/vol1/run/ERR104/ERR10447791/...,ERA18565060,cellular organisms; Bacteria; Pseudomonadota; ...
836,ERR10447792,MinION sequencing,562,OXFORD_NANOPORE,0,ERX9972174,ena-EXPERIMENT-TAB-01-11-2022-07:51:02:528-10529,MinION,SC419,NaN,...,E. coli methylation,Escherichia coli,NaN,NaN,NaN,OXFORDNANOPORE_NATIVE,8465641024,ftp.sra.ebi.ac.uk/vol1/run/ERR104/ERR10447792/...,ERA18565060,cellular organisms; Bacteria; Pseudomonadota; ...
837,ERR10447796,MinION sequencing,562,OXFORD_NANOPORE,0,ERX9972178,ena-EXPERIMENT-TAB-01-11-2022-07:51:02:532-10537,MinION,SC419,NaN,...,E. coli methylation,Escherichia coli,NaN,NaN,NaN,OXFORDNANOPORE_NATIVE,32147641186,ftp.sra.ebi.ac.uk/vol1/run/ERR104/ERR10447796/...,ERA18565060,cellular organisms; Bacteria; Pseudomonadota; ...
838,ERR10447797,MinION sequencing,562,OXFORD_NANOPORE,0,ERX9972179,ena-EXPERIMENT-TAB-01-11-2022-07:51:02:533-10539,MinION,SC452,NaN,...,E. coli methylation,Escherichia coli,NaN,NaN,NaN,OXFORDNANOPORE_NATIVE,15631095638,ftp.sra.ebi.ac.uk/vol1/run/ERR104/ERR10447797/...,ERA18565060,cellular organisms; Bacteria; Pseudomonadota; ...
839,ERR10447803,MinION sequencing,562,OXFORD_NANOPORE,0,ERX9972185,ena-EXPERIMENT-TAB-01-11-2022-07:51:02:534-10551,MinION,SC452,NaN,...,E. coli methylation,Escherichia coli,NaN,NaN,NaN,OXFORDNANOPORE_NATIVE,30720455063,ftp.sra.ebi.ac.uk/vol1/run/ERR104/ERR10447803/...,ERA18565060,cellular organisms; Bacteria; Pseudomonadota; ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34829,ERR10466884,MinION sequencing,309800,OXFORD_NANOPORE,0,ERX9988744,ena-EXPERIMENT-TAB-07-11-2022-16:32:01:911-29198,MinION,NaN,NaN,...,rRNA processing in Archaea using Nanopore-base...,Haloferax volcanii DS2,NaN,NaN,NaN,OXFORDNANOPORE_NATIVE,7931920929,ftp.sra.ebi.ac.uk/vol1/run/ERR104/ERR10466884/...,ERA18577947,cellular organisms; Archaea; Euryarchaeota; St...
34830,ERR10466885,MinION sequencing,2285,OXFORD_NANOPORE,0,ERX9988745,ena-EXPERIMENT-TAB-07-11-2022-16:32:01:911-29200,MinION,NaN,NaN,...,rRNA processing in Archaea using Nanopore-base...,Sulfolobus acidocaldarius,NaN,NaN,NaN,OXFORDNANOPORE_NATIVE,2981578577,ftp.sra.ebi.ac.uk/vol1/run/ERR104/ERR10466885/...,ERA18577947,cellular organisms; Archaea; TACK group; Therm...
34833,ERR10802855,MinION sequencing,309800,OXFORD_NANOPORE,0,ERX10251337,ena-EXPERIMENT-TAB-24-01-2023-06:59:00:004-985,MinION,NaN,Libraries for PCR-cDNA sequencing were prepare...,...,rRNA processing in Archaea using Nanopore-base...,Haloferax volcanii DS2,NaN,NaN,NaN,OXFORDNANOPORE_NATIVE,10484600688,ftp.sra.ebi.ac.uk/vol1/run/ERR108/ERR10802855/...,ERA20292266,cellular organisms; Archaea; Euryarchaeota; St...
34834,ERR11254698,MinION sequencing,309800,OXFORD_NANOPORE,0,ERX10662907,ena-EXPERIMENT-TAB-18-04-2023-18:55:11:693-21786,MinION,NaN,NaN,...,rRNA processing in Archaea using Nanopore-base...,Haloferax volcanii DS2,NaN,NaN,NaN,OXFORDNANOPORE_NATIVE,2898849344,ftp.sra.ebi.ac.uk/vol1/run/ERR112/ERR11254698/...,ERA22056836,cellular organisms; Archaea; Euryarchaeota; St...


In [8]:
df['lineage_save'] = df['lineage'].copy()

In [9]:
def fix_lineage(lineage):
    if lineage is None:
        return None
    l = []
    for lvl in lineage.split(';'):
        lvl = lvl.strip()
        if lvl == "cellular organisms" or 'group' in lvl:
            pass
        else:
            l.append(lvl)
    return "; ".join(l)
df['lineage'] = df['lineage_save'].apply(fix_lineage)

In [10]:
# only keep entries with sufficient lineage information
#
df = df.loc[(~df.lineage.isnull()) & (df.lineage.str.split(';').str.len() > 4)]
print(len(df), "entries with sufficient lineage information")

933 entries with sufficient lineage information


In [11]:
def plot_lineage(df, lvls=6):
    T = nx.DiGraph()
    for lvl in range(0,lvls):
        for val,cnt in df.lineage.str.split(';').str[lvl].value_counts().items():
            if val.strip().replace(" group", ""):
                T.add_node(val.strip().replace(" group", ""), size=cnt)
    for lvl in range(0,lvls-1):
        for edge,cnt in df.lineage.str.split(';').str[lvl:lvl+2].value_counts().items():
            if len(edge) == 2 and None not in edge and " " not in edge:
                u,v = edge
                T.add_edge(u.strip().replace(" group", ""), v.strip().replace(" group", ""))

    pos = graphviz_layout(T, prog="dot", args='-Grankdir="LR"')

    next_nodes = set()
    for v in list(T):
        if not T.out_edges(v):
            for u,v in T.in_edges(v):
                next_nodes.add(u)
    while len(next_nodes) > 0:
        next_nodes_ = set()
        for u in next_nodes:
            new_y = np.median([pos[v][1] for u,v in T.out_edges(u)])
            pos[u] = (pos[u][0], new_y)
            for n,_ in T.in_edges(u):
                next_nodes_.add(n)
        next_nodes = next_nodes_
    for v in ['Archaea','Bacteria']:
        if v in pos:
            pos[v] = (pos[v][0]-90, pos[v][1])
    fig,ax = plt.subplots(figsize=(16, 14))
    nx.draw_networkx(
        T,
        pos,
        with_labels=False,
        node_size=1,
        node_color="white",
        font_size=11,
        font_color="Black",
        ax=ax,
        connectionstyle='angle',
    )
    pos_higher = {}
    for k, v in pos.items():
        pos_higher[k] = (v[0]-140, v[1]+14)
    nx.draw_networkx(
        T,
        pos_higher,
        edgelist=[],
        with_labels=True,
        node_size=0,
        node_color="white",
        font_size=11,
        font_color="Black",
        ax=ax,
        connectionstyle='angle',
    )
    nx.draw_networkx(
        T,
        pos,
        with_labels=True,
        node_size=np.array([int(i) for i in nx.get_node_attributes(T, "size").values()])+200,
        labels=nx.get_node_attributes(T, "size"),
        node_color="Grey",
        edgelist=[],
        font_size=11,
        font_color="White",
        ax=ax,
    )
    ax.set_title("ENA Datasets with ONT raw data")
    plt.show()
plot_lineage(df)

ImportError: requires pygraphviz http://pygraphviz.github.io/

In [ ]:
# select non-amplified samples
sel = (df.library_selection.isin(['RANDOM', 'other', 'unspecified', 'size fractionation'])) & \
      (df.library_source == 'GENOMIC') & \
      (df.library_strategy.isin(['WGS', 'OTHER']))

In [ ]:
GBs = df.submitted_bytes.astype('float') / 10**9
fig,ax = plt.subplots(figsize=(4,3))
ax.hist(GBs, range=(0,200), bins=100)
ax.set(xlabel="GB of data", ylabel="# runs per bin")
plt.show()

In [ ]:
sel &= (GBs > 1) & (GBs < 200)

In [ ]:
df.loc[sel].scientific_name.value_counts()

In [ ]:
frequent_sns = df.loc[sel].scientific_name.value_counts().loc[df.loc[sel].scientific_name.value_counts() > 3]
non_frequent_sns = df.loc[sel].scientific_name.value_counts().loc[df.loc[sel].scientific_name.value_counts() <= 3]
frequent_sns.index

In [ ]:
# hand-select
pd.set_option('display.max_rows', None)
for sn in frequent_sns.index:
    display(df.loc[sel & (df.scientific_name == sn)].sort_values(['study_title','submitted_bytes'], ascending=False)[[
        'run_accession','study_title', 'project_name', 'sample_title', 'sample_alias', 'sample_description', 
        'lineage', 'isolate', 'submitted_bytes', 'submitted_ftp']])
pd.set_option('display.max_rows', 20)

In [ ]:
manual_sel = ['ERR7850327','ERR1474981', 'ERR4699901', 'ERR3485550', 'ERR5191772', 'ERR5353275', 'ERR2868914', 'ERR9793808',
              'ERR9951991', 'ERR2603589', 'ERR4173917', 'ERR10447806', 'ERR5353220', 'ERR1313294', 'ERR9793809', 
              'ERR3654117', 'ERR5243236', 'ERR5295516', 
              'ERR2503988', 'ERR2504003', 'ERR2503999',
              'ERR968973', 'ERR1674581', 'ERR2137852',
              'ERR3654115', 'ERR5353121', 'ERR5295497',
              'ERR5420993', 'ERR5295524', 'ERR5420946',
              'ERR5205861', 'ERR5205857',
              'ERR1309545', 
              'ERR5159245', 'ERR5859545',
              'ERR4173916', 'ERR4179765',
              'ERR5353153', 'ERR5353180',
              'ERR11027274', 'ERR11027272',
              'ERR7477323', 'ERR7477284',
              'ERR3798589', 'ERR3798592', 
              'ERR4656970', 'ERR4656971',
              ]
automatic_sel = df.loc[sel & df.scientific_name.isin(non_frequent_sns.index), 'run_accession'].to_list()

In [ ]:
sel = df.run_accession.isin(manual_sel) | df.run_accession.isin(automatic_sel)

In [ ]:
#ds_per_scientific_name = 3
#sel_sn = []
#for sn in df.loc[sel].scientific_name.unique():
#    #sel_sn.extend(df.loc[df.scientific_name == sn].index[:ds_per_scientific_name]) # just take first
#    sel_sn.extend(df.loc[df.scientific_name == sn].sort_values('submitted_bytes', ascending=False).index[:ds_per_scientific_name]) # select largest dataset
#sel &= df.index.isin(sel_sn)

In [ ]:
plot_lineage(df.loc[sel], lvls=6)

In [ ]:
GBs.loc[sel].sum()

In [ ]:
sel.sum(), len(df.loc[sel].scientific_name.unique())

In [ ]:
df.loc[:, 'dataset'] = 'ENA'
df.loc[:, 'sample'] = df['run_accession']
df.loc[:, 'priority'] = 0
df.loc[:, 'NAT_source'] = df['submitted_ftp']
df.loc[:, 'ref_source'] = ''
df.loc[:, 'WGA_source'] = None

In [32]:
df.loc[sel,
    ['dataset', 'sample', 'NAT_source', 'ref_source', 'WGA_source', 'priority', 
     'study_accession', 'experiment_accession', 'sample_accession', 'run_accession', 'submission_accession',
     #'experiment_title', 'instrument_platform', 'base_count', 'experiment_alias', 'protocol_label', 'pcr_isolation_protocol', 'sequencing_method', 
     'study_title', 'project_name', 'sample_title', 'sample_alias', 'sample_description', # 'sample_material', 'run_alias', 
     'tax_id', 'scientific_name', 'lineage', 'isolate', #'taxonomic_identity_marker',  'sub_strain', 'taxonomic_classification',
     'library_construction_protocol', 'library_layout', 'library_name', #'library_selection', 'library_source', 'library_strategy', #'library_gen_protocol',
     'instrument_model',  
     'submitted_bytes', 'submitted_ftp', #'sra_bytes', 'sra_ftp', 'read_count','submitted_format', 
    ]].to_csv("../datasets/ENA_dataset.csv")

In [37]:
! ls "../datasets/"

ENA		 MS061_lactate	No684_Glaeser_Mikroplastik
ENA_dataset.csv  MS061_nitrate	Veillonella.zip
MS061_control	 MicrobeMod	other
